# First Model

In this notebook, we create a simple model using LightGBM. The features included in this model are:
- all float (or int but not category) variables as it is:
    - `RevLineCr`, `NoEmp`, `CreateJob`, `RetainedJob`, `ApprovalFY`. `DisbursementGross`, `GrAppv`, `SBA_Appv`
- some categorical variables as it is:
    - `NewExist`, `RevLineCr`, `LowDoc`, `UrbanRural`
- Some date objects as daystamp:
    - `DisbursementDate_daystamp`, `ApprovalDate_daystamp`
- Some categorical varibles with coarse labeling:
    - `FranchiseCode`(0,1,or others)
- Some categorical variables with holdout target encoding:
    - `Sector`, `State`, `BankState`
- `Longitude`, `Latitude`: holdout target encoded with `HistGradientBoostingClassifier`

Note that `City` is not used in this model

In [2]:
import pandas as pd
import numpy as np
from cat_encodings import target_encode_test, target_encode_smooth_test
from clean_data import clean_data
import os

## データの準備

In [3]:
DATA_DIR = "../data"
EDITED_DATA_DIR = "edited_data"

In [4]:
# load data
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"), index_col = 0)
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"), index_col = 0)
geo_train = pd.read_csv(os.path.join(EDITED_DATA_DIR, "train_geohash.csv"), index_col = 0)
geo_test = pd.read_csv(os.path.join(EDITED_DATA_DIR, "test_geohash.csv"), index_col = 0)

In [5]:
# clean data
train = clean_data(train)
test = clean_data(test)

In [6]:
# columns that are in geo_train but not train
print(set(geo_train) - set(train))

{'longitude', 'origin', 'latitude', 'location', 'geohash'}


In [7]:
# we use latitude and longitude col
for col in ['latitude', 'longitude']:
    train[col] = geo_train[col]
    test[col] = geo_test[col]

In [8]:
train.dtypes

Term                                  int64
NoEmp                                 int64
NewExist                           category
CreateJob                             int64
RetainedJob                           int64
FranchiseCode                      category
RevLineCr                          category
LowDoc                             category
DisbursementDate             datetime64[ns]
MIS_Status                            int64
Sector                             category
ApprovalDate                 datetime64[ns]
ApprovalFY                            int64
City                                 object
State                                object
BankState                            object
DisbursementGross                   float64
GrAppv                              float64
SBA_Appv                            float64
UrbanRural                         category
DisbursementDate_year               float64
DisbursementDate_month              float64
DisbursementDate_day            

In [9]:
# columns used for training -> all_cols
num_cols = ['NoEmp', 'CreateJob', 'RetainedJob', 'ApprovalFY', 'DisbursementGross', 'GrAppv', 'SBA_Appv']
retained_cat_cols = ['NewExist', 'RevLineCr', 'LowDoc', 'UrbanRural']
timestamp_cols = ['DisbursementDate_daystamp', 'ApprovalDate_daystamp']
franchise_cols = ['FranchiseCode1', 'FranchiseCode0']
target_encode_cols = ['Sector', 'State', 'BankState']
target_encoded_cols = [item + "_target" for item in target_encode_cols]
target_encode_smooth_cols = ["longitude", "latitude"]
target_encoded_smooth_cols = [item + "_target" for item in target_encode_smooth_cols]
location_cols = ['latitude', 'longitude']
all_cols = num_cols + retained_cat_cols + timestamp_cols + franchise_cols + target_encoded_cols + location_cols + target_encoded_smooth_cols

## train, val の分割

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(train,train["MIS_Status"], test_size=0.2, random_state=42, stratify=train['MIS_Status']) # stratifyした方がいいかも

### target encoding for valid data

trainの値を使って，testのtarget encodingをします．

In [12]:
from sklearn.model_selection import StratifiedKFold

In [13]:
target_encode_cols = ['Sector', 'State', 'BankState']
    
# We can simply use training data to encode the test data
for col in target_encode_cols:
    X_val = target_encode_test(X_train, y_train, X_val, col)

In [14]:
# for smooth data:
X_val =target_encode_smooth_test(X_train, y_train, X_val, "longitude")
X_val =target_encode_smooth_test(X_train, y_train, X_val, "latitude")

### Target encoding for train data

CVの分け方と合うように，target encodingをしていきます．

In [15]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1000)
# for train
kf_iter_train = kf.split(X_train, y_train)

# create list of indices for training / test data to use for holdout target encoding
folds_train = []
for train_idx, test_idx in kf_iter_train:
    folds_train.append((train_idx, test_idx))

In [16]:
X_trs = []; X_cvalids = []
for fold, (train_indices, cvalid_indices) in enumerate(folds_train):
    X_tr, X_cvalid = X_train.iloc[train_indices], X_train.iloc[cvalid_indices]
    y_tr, y_cvalid = y_train.iloc[train_indices], y_train.iloc[cvalid_indices]
    for col in target_encode_cols:
        X_tr = target_encode_test(X_cvalid, y_cvalid, X_tr, col)
        X_cvalid = target_encode_test(X_tr, y_tr, X_cvalid, col)
    for col in target_encode_smooth_cols:
        X_tr = target_encode_smooth_test(X_cvalid, y_cvalid, X_tr, col)
        X_cvalid = target_encode_smooth_test(X_tr, y_tr, X_cvalid, col)
    X_trs.append(X_tr[all_cols])
    X_cvalids.append(X_cvalid[all_cols])

/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = train_X; df['target'] = train_y
/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X.loc[:, column + "_target"] = 0.9
/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:52: SettingWithCopyWarning: 
A value is trying to 

## LightGBM training

### Macro F1 Score (with threshold included in the metric)

In [17]:
from sklearn.metrics import  f1_score
def Macrof1(preds, eval_dataset):
    y_true = eval_dataset.get_label()
    max_score =0
    for th in np.linspace(0.2,0.9,100):
        y_pred = (preds>th).astype(int)
        score = f1_score(y_true, y_pred, average='macro')
        if score > max_score:
            max_score = score
    return 'Macrof1', max_score, True

### CV Training!

In [18]:
valid_scores = []
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore", category=DeprecationWarning) 
boosters = []
for fold, (train_indices, cvalid_indices) in enumerate(folds_train):
    X_tr = X_trs[fold]
    X_cvalid = X_cvalids[fold]
    y_tr = y_train.iloc[train_indices]
    y_cvalid = y_train.iloc[cvalid_indices]
    
    lgb_train = lgb.Dataset(X_tr[all_cols], y_tr)
    lgb_eval = lgb.Dataset(X_cvalid[all_cols], y_cvalid)

    print("fold No. = ", fold)
    params = {
    'objective': 'binary',
    'metric': 'None',  # Use custom to use the custom metric for evaluation
    'verbose': -1,
    'learning_rate':0.1,
    'early_stopping_rounds': 100,
    'scale_pos_weight': 1.0,
    }
    
    model = lgb.train(
        params,
        lgb_train,
        valid_sets = lgb_eval,
        num_boost_round = 100,
        feval = Macrof1,
        callbacks=[lgb.log_evaluation(10)],
    )
    
    print("now caluculating MacroF1 values .....")
    name, score, _ = Macrof1(model.predict(X_cvalid), lgb_eval)
    print(f'fold {fold} MacroF1: {score}')
    valid_scores.append(score)

    boosters.append(model)

fold No. =  0
[10]	valid_0's Macrof1: 0.675214
[20]	valid_0's Macrof1: 0.675026
[30]	valid_0's Macrof1: 0.679535
[40]	valid_0's Macrof1: 0.679372
[50]	valid_0's Macrof1: 0.679163
[60]	valid_0's Macrof1: 0.679484
[70]	valid_0's Macrof1: 0.677288
[80]	valid_0's Macrof1: 0.678234
[90]	valid_0's Macrof1: 0.675831
[100]	valid_0's Macrof1: 0.675196
now caluculating MacroF1 values .....
fold 0 MacroF1: 0.6811394737030745
fold No. =  1
[10]	valid_0's Macrof1: 0.67943
[20]	valid_0's Macrof1: 0.678517
[30]	valid_0's Macrof1: 0.683826
[40]	valid_0's Macrof1: 0.682103
[50]	valid_0's Macrof1: 0.682887
[60]	valid_0's Macrof1: 0.684089
[70]	valid_0's Macrof1: 0.683948
[80]	valid_0's Macrof1: 0.683984
[90]	valid_0's Macrof1: 0.683202
[100]	valid_0's Macrof1: 0.684942
now caluculating MacroF1 values .....
fold 1 MacroF1: 0.685949890781707
fold No. =  2
[10]	valid_0's Macrof1: 0.671947
[20]	valid_0's Macrof1: 0.67342
[30]	valid_0's Macrof1: 0.675594
[40]	valid_0's Macrof1: 0.674858
[50]	valid_0's Macrof

In [19]:
valid_scores

[0.6811394737030745,
 0.685949890781707,
 0.6773881520367102,
 0.6774831551115094,
 0.6850199021960222]

### Check scores with valid data

In [20]:
pred_average = np.mean([item.predict(X_val[all_cols]) for item in boosters], axis=0)
test_prediction = (pred_average > np.quantile(pred_average, 0.1)).astype(int) # ここはもう少し良い選び方があるはずです．

In [21]:
f1_score(y_val, test_prediction, average='macro')

0.6748269215274292

## LGB training with all data

### target encoding for CV

In [22]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1000)
# for train
kf_iter_train = kf.split(train, train['MIS_Status'])

# create list of indices for training / test data to use for holdout target encoding
folds_train = []
for train_idx, test_idx in kf_iter_train:
    folds_train.append((train_idx, test_idx))

In [23]:
X_trs = []; X_cvalids = []
for fold, (train_indices, cvalid_indices) in enumerate(folds_train):
    X_tr, X_cvalid = train.iloc[train_indices], train.iloc[cvalid_indices]
    y_tr, y_cvalid = train['MIS_Status'].iloc[train_indices], train['MIS_Status'].iloc[cvalid_indices]
    for col in target_encode_cols:
        X_tr = target_encode_test(X_cvalid, y_cvalid, X_tr, col)
        X_cvalid = target_encode_test(X_tr, y_tr, X_cvalid, col)
    for col in target_encode_smooth_cols:
        X_tr = target_encode_smooth_test(X_cvalid, y_cvalid, X_tr, col)
        X_cvalid = target_encode_smooth_test(X_tr, y_tr, X_cvalid, col)
    X_trs.append(X_tr[all_cols])
    X_cvalids.append(X_cvalid[all_cols])

/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = train_X; df['target'] = train_y
/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X.loc[:, column + "_target"] = 0.9
/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/notebooks/cat_encodings.py:52: SettingWithCopyWarning: 
A value is trying to 

In [24]:
valid_scores = []
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore", category=DeprecationWarning) 
boosters = []
for fold, (train_indices, cvalid_indices) in enumerate(folds_train):
    X_tr = X_trs[fold]
    X_cvalid = X_cvalids[fold]
    y_tr = train['MIS_Status'].iloc[train_indices]
    y_cvalid = train['MIS_Status'].iloc[cvalid_indices]
    
    lgb_train = lgb.Dataset(X_tr[all_cols], y_tr)
    lgb_eval = lgb.Dataset(X_cvalid[all_cols], y_cvalid)

    print("fold No. = ", fold)
    params = {
    'objective': 'binary',
    'metric': 'None',  # Use custom to use the custom metric for evaluation
    'verbose': -1,
    'learning_rate':0.1,
    'early_stopping_rounds': 100,
    'scale_pos_weight': 1.0,
    }
    
    model = lgb.train(
        params,
        lgb_train,
        valid_sets = lgb_eval,
        num_boost_round = 100,
        feval = Macrof1,
        callbacks=[lgb.log_evaluation(10)],
    )
    
    print("now caluculating MacroF1 values .....")
    name, score, _ = Macrof1(model.predict(X_cvalid), lgb_eval)
    print(f'fold {fold} MacroF1: {score}')
    valid_scores.append(score)

    boosters.append(model)

fold No. =  0
[10]	valid_0's Macrof1: 0.664959
[20]	valid_0's Macrof1: 0.670974
[30]	valid_0's Macrof1: 0.670066
[40]	valid_0's Macrof1: 0.668976
[50]	valid_0's Macrof1: 0.671069
[60]	valid_0's Macrof1: 0.671059
[70]	valid_0's Macrof1: 0.669541
[80]	valid_0's Macrof1: 0.668194
[90]	valid_0's Macrof1: 0.669673
[100]	valid_0's Macrof1: 0.667827
now caluculating MacroF1 values .....
fold 0 MacroF1: 0.67213969338605
fold No. =  1
[10]	valid_0's Macrof1: 0.682413
[20]	valid_0's Macrof1: 0.685429
[30]	valid_0's Macrof1: 0.689238
[40]	valid_0's Macrof1: 0.686696
[50]	valid_0's Macrof1: 0.685768
[60]	valid_0's Macrof1: 0.683535
[70]	valid_0's Macrof1: 0.684584
[80]	valid_0's Macrof1: 0.686696
[90]	valid_0's Macrof1: 0.686974
[100]	valid_0's Macrof1: 0.6883
now caluculating MacroF1 values .....
fold 1 MacroF1: 0.6892383715376068
fold No. =  2
[10]	valid_0's Macrof1: 0.671439
[20]	valid_0's Macrof1: 0.67698
[30]	valid_0's Macrof1: 0.680582
[40]	valid_0's Macrof1: 0.679697
[50]	valid_0's Macrof1:

In [25]:
valid_scores

[0.67213969338605,
 0.6892383715376068,
 0.6820775227378263,
 0.6789978338773986,
 0.6775739461249936]

### prepare submission
#### target encoding for test data

In [26]:
target_encode_cols = ['Sector', 'State', 'BankState']
    
# We can simply use training data to encode the test data
for col in target_encode_cols:
    test = target_encode_test(train, train['MIS_Status'], test, col)
# for smooth data:
test =target_encode_smooth_test(train, train['MIS_Status'], test, "longitude")
test =target_encode_smooth_test(train, train['MIS_Status'], test, "latitude")

In [27]:
# prepare submission
pred_average = np.mean([item.predict(test[all_cols]) for item in boosters], axis=0)
test_prediction = (pred_average > np.quantile(pred_average, 0.1)).astype(int) # ここはもう少し良い選び方があるはずです．

In [28]:
test['prediction'] = test_prediction
test['prediction'].to_csv('submission_ex.csv', header=None)